<a href="https://colab.research.google.com/github/Madiha64/Madiha-CV-cat-dog-cnn/blob/main/DCgan_mnist.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [1]:
import tensorflow as tf
from tensorflow.keras import layers, models
import tensorflow_datasets as tfds
import matplotlib.pyplot as plt
from tensorflow.keras.layers import Dense, Input
from tensorflow.keras.layers import Conv2D, Flatten
from tensorflow.keras.layers import Reshape, Conv2DTranspose
from tensorflow.keras.models import Model
from tensorflow.keras import backend as K
from tensorflow.keras.datasets import mnist
from tensorflow.keras.layers import LeakyReLU
import numpy as np
import matplotlib.pyplot as plt
from PIL import Image
from tensorflow.keras.layers import BatchNormalization
def build_and_train_models():
  (x_train, _), (_, _) = mnist.load_data()# load MNIST dataset
# reshape data for CNN as (28, 28, 1) and normalize
  image_size = x_train.shape[1]
  x_train = np.reshape(x_train, [-1, image_size, image_size, 1])
  x_train = x_train.astype('float32') / 255
  model_name = "dcgan_mnist"   # network parameters
# the latent or z vector is 100-dim
  latent_size = 100
  batch_size = 64
  train_steps = 1000 # you can change the steps
  learning_rate = 2e-4
  decay = 6e-8
# build discriminator model
  input_shape = (image_size, image_size, 1)
  inputs = Input(shape=input_shape, name='discriminator_input')
  discriminator = build_discriminator(inputs)
# [1] or original paper uses Adam,
# but discriminator converges easily with RMSprop
  optimizer = RMSprop(learning_rate=learning_rate, decay=decay)
  discriminator.compile(loss='binary_crossentropy',
                        optimizer=optimizer,metrics=['accuracy'])
  discriminator.summary()
# build generator model
  input_shape = (latent_size, )
  inputs = Input(shape=input_shape, name='z_input')
  generator = build_generator(inputs, image_size)
  generator.summary()
# build adversarial model
  optimizer = RMSprop(learning_rate=learning_rate * 0.5, decay=decay * 0.5)
# freeze the weights of discriminator during adversarial training
  discriminator.trainable = False
# adversarial = generator + discriminator
  adversarial = Model(inputs, discriminator(generator(inputs)),name=model_name)
  adversarial.compile(loss='binary_crossentropy',optimizer=optimizer,metrics=['accuracy'])
  adversarial.summary()
# train discriminator and adversarial networks
  models = (generator, discriminator, adversarial)
  params = (batch_size, latent_size, train_steps, model_name)
  train(models, x_train, params)

In [2]:
def build_discriminator(inputs):
  kernel_size = 5
  layer_filters = [32, 64, 128, 256]  # first 3 convolution layers use strides = 2
  x = inputs                                     # last one uses strides = 1
  for filters in layer_filters:
    if filters == layer_filters[-1]:
      strides = 1
    else:
      strides = 2
  x = LeakyReLU(alpha=0.2)(x)
  x = Conv2D(filters=filters,
              kernel_size=kernel_size,
              strides=strides,
              padding='same')(x)
  x = Flatten()(x)
  x = Dense(1)(x)
  x = Activation('sigmoid')(x)
  discriminator = Model(inputs, x, name='discriminator')
  return discriminator

In [3]:

from tensorflow.keras.layers import BatchNormalization, Dense, Reshape, Activation, Conv2DTranspose
from tensorflow.keras.models import Model
def build_generator(inputs, image_size):
  image_resize = image_size // 4# network parameters
  kernel_size = 5
  layer_filters = [128, 64, 32, 1]
  x = Dense(image_resize * image_resize * layer_filters[0])(inputs)
  x = Reshape((image_resize, image_resize, layer_filters[0]))(x)
  for filters in layer_filters:
    if filters > layer_filters[-2]:
      strides = 2   # first two convolution layers use
    else:
       strides = 1             # the last two use strides = 1

    x = BatchNormalization()(x)
    x = Activation('relu')(x)
    x = Conv2DTranspose(filters=filters,
                    kernel_size=kernel_size,
                    strides=strides,
                    padding='same')(x)
  x = Activation('sigmoid')(x)
  generator = Model(inputs, x, name='generator')
  return generator

In [4]:
from tensorflow.keras.layers import LeakyReLU
import tensorflow as tf
from tensorflow.keras import layers, models
import tensorflow_datasets as tfds
import matplotlib.pyplot as plt
from tensorflow.keras.layers import Dense, Input
from tensorflow.keras.layers import Conv2D, Flatten
from tensorflow.keras.layers import Reshape, Conv2DTranspose
from tensorflow.keras.models import Model
from tensorflow.keras import backend as K
from tensorflow.keras.datasets import mnist
import numpy as np
import matplotlib.pyplot as plt
from PIL import Image
from tensorflow.keras.optimizers import RMSprop
from tensorflow.keras.layers import BatchNormalization
def train(models, x_train, params):# the GAN component models
  generator, discriminator, adversarial = models # network parameters
  batch_size, latent_size, train_steps, model_name = params
# the generator image is saved every 500 steps
  save_interval = 10
  # noise vector to see how the generator output evolves during training
  noise_input = np.random.uniform(-1.0, 1.0, size=[16, latent_size])
# number of elements in train dataset
  train_size = x_train.shape[0]
  for i in range(train_steps):# train the discriminator for 1 batch
# 1 batch of real (label=1.0) and fake images (label=0.0)
# randomly pick real images from dataset
      rand_indexes = np.random.randint(0, train_size, size=batch_size)
      real_images = x_train[rand_indexes]
# generate fake images from noise using generator
# generate noise using uniform distribution
      noise = np.random.uniform(-1.0,1.0,size=[batch_size, latent_size])
# generate fake images
      fake_images = generator.predict(noise)
# real + fake images = 1 batch of train data
      x = np.concatenate((real_images, fake_images))
# label real and fake images
# real images label is 1.0
      y = np.ones([2 * batch_size, 1])
# fake images label is 0.0
      y[batch_size:, :] = 0.0
# train discriminator network, log the loss and accuracy
      loss, acc = discriminator.train_on_batch(x, y)
      log = "%d: [discriminator loss: %f, acc: %f]" % (i, loss, acc)
# train the adversarial network for 1 batch
# 1 batch of fake images with label=1.0
# since the discriminator weights
# are frozen in adversarial network
# only the generator is trained
# generate noise using uniform distribution
      noise = np.random.uniform(-1.0,1.0,size=[batch_size, latent_size])
# label fake images as real or 1.0
      y = np.ones([batch_size, 1])
# train the adversarial network
# note that unlike in discriminator training,
# we do not save the fake images in a variable
# the fake images go to the discriminator input of the adversarial
# for classification
# log the loss and accuracy
      loss, acc = adversarial.train_on_batch(noise, y)  # plot generator images on a periodic basis
      log = "%s [adversarial loss: %f, acc: %f]" % (log, loss, acc)
      print(log)
     # if (i + 1) % save_interval == 0:
        #plot_images(generator,noise_input=noise_input,show=True,step=(i + 1),model_name=model_name)
      if (i + 1) % save_interval == 0:
        try:
          plot_images(generator, noise_input=noise_input, show=True, step=(i + 1), model_name=model_name)
        except Exception as e:
         print(f"[Step {i+1}] Error while plotting images: {e}")

# save the model after training the generator
# the trained generator can be reloaded for
# future MNIST digit generation
      generator.save(model_name + ".h5")
build_and_train_models()


11490434/11490434 ━━━━━━━━━━━━━━━━━━━━ 0s 0us/step


/usr/local/lib/python3.11/dist-packages/keras/src/layers/activations/leaky_relu.py:41: UserWarning: Argument `alpha` is deprecated. Use `negative_slope` instead.
  warnings.warn(
/usr/local/lib/python3.11/dist-packages/keras/src/optimizers/base_optimizer.py:86: UserWarning: Argument `decay` is no longer supported and will be ignored.
  warnings.warn(


Model: "discriminator"

┏━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━┳━━━━━━━━━━━━━━━━━━━━━━━━┳━━━━━━━━━━━━━━━┓
┃ Layer (type)                    ┃ Output Shape           ┃       Param # ┃
┡━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━╇━━━━━━━━━━━━━━━━━━━━━━━━╇━━━━━━━━━━━━━━━┩
│ discriminator_input             │ (None, 28, 28, 1)      │             0 │
│ (InputLayer)                    │                        │               │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ leaky_re_lu (LeakyReLU)         │ (None, 28, 28, 1)      │             0 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ conv2d (Conv2D)                 │ (None, 28, 28, 256)    │         6,656 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ flatten (Flatten)               │ (None, 200704)         │             0 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ dense (Dense)                   │ (None, 1)              │       200,705 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ activation (Activation)         │ (None, 1)              │             0 │
└─────────────────────────────────┴────────────────────────┴───────────────┘

 Total params: 207,361 (810.00 KB)

...

995: [discriminator loss: 0.985706, acc: 0.207471] [adversarial loss: 0.301599, acc: 0.999922]
2/2 ━━━━━━━━━━━━━━━━━━━━ 0s 198ms/step


996: [discriminator loss: 0.985839, acc: 0.207450] [adversarial loss: 0.301539, acc: 0.999922]
2/2 ━━━━━━━━━━━━━━━━━━━━ 0s 208ms/step


997: [discriminator loss: 0.985972, acc: 0.207430] [adversarial loss: 0.301480, acc: 0.999922]
2/2 ━━━━━━━━━━━━━━━━━━━━ 1s 312ms/step


998: [discriminator loss: 0.986105, acc: 0.207418] [adversarial loss: 0.301420, acc: 0.999922]
2/2 ━━━━━━━━━━━━━━━━━━━━ 0s 199ms/step


999: [discriminator loss: 0.986235, acc: 0.207422] [adversarial loss: 0.301361, acc: 0.999922]
[Step 1000] Error while plotting images: name 'plot_images' is not defined


In [5]:
def plot_images(generator, noise_input, show=True, step=0, model_name="model"):
    try:
        images = generator.predict(noise_input)
        plt.figure(figsize=(4, 4))
        for i in range(images.shape[0]):
            plt.subplot(4, 4, i + 1)
            img = images[i, :, :, 0]
            plt.imshow(img, cmap='gray')
            plt.axis('off')
        plt.tight_layout()
        if show:
            plt.show()
        else:
            filename = f"{model_name}_step_{step}.png"
            plt.savefig(filename)
            plt.close()
    except Exception as e:
        print(f"plot_images() failed: {e}")
